In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn import metrics
import torch.optim as optim

In [2]:
sub_features = ['42 tGravityAcc-mean()-Y',
 '43 tGravityAcc-mean()-Z',
 '51 tGravityAcc-max()-Y',
 '52 tGravityAcc-max()-Z',
 '54 tGravityAcc-min()-Y',
 '55 tGravityAcc-min()-Z',
 '56 tGravityAcc-sma()',
 '59 tGravityAcc-energy()-Z',
 '125 tBodyGyro-std()-Y',
 '128 tBodyGyro-mad()-Y',
 '138 tBodyGyro-energy()-Y',
 '165 tBodyGyroJerk-std()-Y',
 '168 tBodyGyroJerk-mad()-Y',
 '178 tBodyGyroJerk-energy()-Y',
 '181 tBodyGyroJerk-iqr()-Y',
 '425 fBodyGyro-mean()-Y',
 '428 fBodyGyro-std()-Y',
 '431 fBodyGyro-mad()-Y',
 '441 fBodyGyro-energy()-Y',
 '475 fBodyGyro-bandsEnergy()-1,8',
 '478 fBodyGyro-bandsEnergy()-25,32',
 '483 fBodyGyro-bandsEnergy()-1,16',
 '487 fBodyGyro-bandsEnergy()-1,24',
 '559 angle(X,gravityMean)',
 '560 angle(Y,gravityMean)',
 '561 angle(Z,gravityMean)']

act_features = ['4 tBodyAcc-std()-X',
 '7 tBodyAcc-mad()-X',
 '10 tBodyAcc-max()-X',
 '17 tBodyAcc-energy()-X',
 '202 tBodyAccMag-std()',
 '204 tBodyAccMag-max()',
 '215 tGravityAccMag-std()',
 '217 tGravityAccMag-max()',
 '266 fBodyAcc-mean()-X',
 '269 fBodyAcc-std()-X',
 '272 fBodyAcc-mad()-X',
 '275 fBodyAcc-max()-X',
 '282 fBodyAcc-energy()-X',
 '303 fBodyAcc-bandsEnergy()-1,8',
 '311 fBodyAcc-bandsEnergy()-1,16',
 '315 fBodyAcc-bandsEnergy()-1,24',
 '504 fBodyAccMag-std()',
 '505 fBodyAccMag-mad()',
 '506 fBodyAccMag-max()',
 '509 fBodyAccMag-energy()']

input_shape = len(sub_features) + len(act_features)

In [3]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = input_shape):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 25),
            classifier_block(25, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = input_shape, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 80),
            generator_block(80, 60),
            generator_block(60, 50),
            nn.Linear(50, feature_dim),
            nn.Tanh()
        )
    def forward(self, noise):
        return self.gen(noise)

def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../../saved_models/{model_name}'))

In [5]:
#label is a list of integers specifying which labels to filter by
#users is a list of integers specifying which users to filter by
#y_label is a string, either "Activity" or "Subject" depending on what y output needs to be returned
def start_data(label, users, y_label, sub_features, act_features):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names
    
    X_train_1 = data[sub_features]
    X_train_2 = data[act_features]
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']
    
    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X_train = GAN_data.iloc[:,:-2].values
    y_train = GAN_data[[y_label]].values
    
    return X_train, y_train.flatten()

# Activity Classifier

In [6]:
activities = [1, 3, 4]
users = [23, 25, 27]

X, y = start_data(activities, users, "Activity", sub_features, act_features)

In [7]:
for k in range(len(y)):
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [9]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 2.2210010290145874, Final Batch Loss: 1.099376916885376
Epoch 2, Loss: 2.218256711959839, Final Batch Loss: 1.1205741167068481
Epoch 3, Loss: 2.2056864500045776, Final Batch Loss: 1.1011734008789062
Epoch 4, Loss: 2.195812940597534, Final Batch Loss: 1.102722406387329
Epoch 5, Loss: 2.1909762620925903, Final Batch Loss: 1.0902897119522095
Epoch 6, Loss: 2.1918030977249146, Final Batch Loss: 1.1062830686569214
Epoch 7, Loss: 2.182931900024414, Final Batch Loss: 1.0880934000015259
Epoch 8, Loss: 2.1788216829299927, Final Batch Loss: 1.0990973711013794
Epoch 9, Loss: 2.1747478246688843, Final Batch Loss: 1.08416748046875
Epoch 10, Loss: 2.172971248626709, Final Batch Loss: 1.1020557880401611
Epoch 11, Loss: 2.155690550804138, Final Batch Loss: 1.0861163139343262
Epoch 12, Loss: 2.1507601737976074, Final Batch Loss: 1.0744158029556274
Epoch 13, Loss: 2.1456964015960693, Final Batch Loss: 1.0695973634719849
Epoch 14, Loss: 2.1345083713531494, Final Batch Loss: 1.0823786258697

Epoch 122, Loss: 0.08123305439949036, Final Batch Loss: 0.03892291709780693
Epoch 123, Loss: 0.12660792469978333, Final Batch Loss: 0.05204271525144577
Epoch 124, Loss: 0.12824956327676773, Final Batch Loss: 0.058128125965595245
Epoch 125, Loss: 0.09176888316869736, Final Batch Loss: 0.042134154587984085
Epoch 126, Loss: 0.11140896379947662, Final Batch Loss: 0.050422199070453644
Epoch 127, Loss: 0.08848652802407742, Final Batch Loss: 0.0625363439321518
Epoch 128, Loss: 0.11621509119868279, Final Batch Loss: 0.06204315647482872
Epoch 129, Loss: 0.12897387146949768, Final Batch Loss: 0.06596853584051132
Epoch 130, Loss: 0.12000427022576332, Final Batch Loss: 0.071745365858078
Epoch 131, Loss: 0.1257382370531559, Final Batch Loss: 0.07757174223661423
Epoch 132, Loss: 0.1260243095457554, Final Batch Loss: 0.059997234493494034
Epoch 133, Loss: 0.10590589791536331, Final Batch Loss: 0.040114715695381165
Epoch 134, Loss: 0.10213616117835045, Final Batch Loss: 0.06846955418586731
Epoch 135, L

Epoch 239, Loss: 0.030637627467513084, Final Batch Loss: 0.015437518246471882
Epoch 240, Loss: 0.026258678175508976, Final Batch Loss: 0.007814292795956135
Epoch 241, Loss: 0.020069904159754515, Final Batch Loss: 0.00678354362025857
Epoch 242, Loss: 0.041482944041490555, Final Batch Loss: 0.025850653648376465
Epoch 243, Loss: 0.016723393462598324, Final Batch Loss: 0.004866982810199261
Epoch 244, Loss: 0.05787427071481943, Final Batch Loss: 0.010320303030312061
Epoch 245, Loss: 0.084104523062706, Final Batch Loss: 0.06320188194513321
Epoch 246, Loss: 0.014277137815952301, Final Batch Loss: 0.0061858948320150375
Epoch 247, Loss: 0.07641522586345673, Final Batch Loss: 0.040133874863386154
Epoch 248, Loss: 0.029842442832887173, Final Batch Loss: 0.009768613614141941
Epoch 249, Loss: 0.019743362441658974, Final Batch Loss: 0.010996597819030285
Epoch 250, Loss: 0.020047073252499104, Final Batch Loss: 0.009019224904477596
Epoch 251, Loss: 0.026421205140650272, Final Batch Loss: 0.01756222918

Epoch 362, Loss: 0.014776397496461868, Final Batch Loss: 0.0052253324538469315
Epoch 363, Loss: 0.025817206129431725, Final Batch Loss: 0.01715395599603653
Epoch 364, Loss: 0.037985384464263916, Final Batch Loss: 0.011669909581542015
Epoch 365, Loss: 0.01944965496659279, Final Batch Loss: 0.009347076527774334
Epoch 366, Loss: 0.0393194577191025, Final Batch Loss: 0.03601067513227463
Epoch 367, Loss: 0.012790870852768421, Final Batch Loss: 0.0070074936375021935
Epoch 368, Loss: 0.03828414902091026, Final Batch Loss: 0.016425058245658875
Epoch 369, Loss: 0.02743233833462, Final Batch Loss: 0.00335076916962862
Epoch 370, Loss: 0.01696638949215412, Final Batch Loss: 0.0069939615204930305
Epoch 371, Loss: 0.034032752737402916, Final Batch Loss: 0.018057778477668762
Epoch 372, Loss: 0.0289319371804595, Final Batch Loss: 0.013755989260971546
Epoch 373, Loss: 0.01574952807277441, Final Batch Loss: 0.010622701607644558
Epoch 374, Loss: 0.019671378657221794, Final Batch Loss: 0.01181786414235830

Epoch 477, Loss: 0.006857580738142133, Final Batch Loss: 0.00295214238576591
Epoch 478, Loss: 0.0247041592374444, Final Batch Loss: 0.011502167209982872
Epoch 479, Loss: 0.01822765450924635, Final Batch Loss: 0.005239257588982582
Epoch 480, Loss: 0.009283947059884667, Final Batch Loss: 0.0013024380896240473
Epoch 481, Loss: 0.009916054550558329, Final Batch Loss: 0.002466723322868347
Epoch 482, Loss: 0.01771247759461403, Final Batch Loss: 0.014177820645272732
Epoch 483, Loss: 0.008775162510573864, Final Batch Loss: 0.004791310988366604
Epoch 484, Loss: 0.020354495849460363, Final Batch Loss: 0.003906428348273039
Epoch 485, Loss: 0.004129496752284467, Final Batch Loss: 0.0015589402755722404
Epoch 486, Loss: 0.015122492564842105, Final Batch Loss: 0.002146404003724456
Epoch 487, Loss: 0.011867280350998044, Final Batch Loss: 0.0030036114621907473
Epoch 488, Loss: 0.01125239534303546, Final Batch Loss: 0.008213710971176624
Epoch 489, Loss: 0.004643642343580723, Final Batch Loss: 0.00346342

...

Epoch 4500, Loss: 8.902773151930887e-05, Final Batch Loss: 7.24334386177361e-05
Epoch 4501, Loss: 0.0005694666378985858, Final Batch Loss: 2.0287194274715148e-05
Epoch 4502, Loss: 0.00018890260435000528, Final Batch Loss: 2.0543129721772857e-05
Epoch 4503, Loss: 9.118013622355647e-05, Final Batch Loss: 6.364922592183575e-05
Epoch 4504, Loss: 3.82817361241905e-05, Final Batch Loss: 1.4499069948215038e-05
Epoch 4505, Loss: 1.2539280987766688e-05, Final Batch Loss: 1.7218656012119027e-06
Epoch 4506, Loss: 7.746948540443555e-05, Final Batch Loss: 5.090756530989893e-05
Epoch 4507, Loss: 9.36816441026167e-05, Final Batch Loss: 5.155002327228431e-06
Epoch 4508, Loss: 6.212228981894441e-05, Final Batch Loss: 8.602954039815813e-06
Epoch 4509, Loss: 9.578599565429613e-05, Final Batch Loss: 3.805156302405521e-05
Epoch 4510, Loss: 1.7846781247499166e-05, Final Batch Loss: 1.4173228009894956e-05
Epoch 4511, Loss: 0.002965089453937253, Final Batch Loss: 5.432433317764662e-05
Epoch 4512, Loss: 0.0020

Epoch 4627, Loss: 0.000768890736708272, Final Batch Loss: 6.640986157435691e-06
Epoch 4628, Loss: 6.268407355491945e-05, Final Batch Loss: 6.019835200277157e-05
Epoch 4629, Loss: 0.00015530511518591084, Final Batch Loss: 0.00010887916141655296
Epoch 4630, Loss: 0.00012586971024575178, Final Batch Loss: 0.00012492801761254668
Epoch 4631, Loss: 5.240341354806333e-05, Final Batch Loss: 5.205866546020843e-05
Epoch 4632, Loss: 1.860378836227028e-05, Final Batch Loss: 1.832860675676784e-06
Epoch 4633, Loss: 1.279352272831602e-05, Final Batch Loss: 2.0819497876800597e-06
Epoch 4634, Loss: 8.327363548232825e-05, Final Batch Loss: 7.842451304895803e-05
Epoch 4635, Loss: 1.8280436506756814e-05, Final Batch Loss: 6.842044513177825e-06
Epoch 4636, Loss: 2.6045906906801974e-05, Final Batch Loss: 2.1249206838547252e-05
Epoch 4637, Loss: 8.902522824882908e-05, Final Batch Loss: 8.790184801910073e-05
Epoch 4638, Loss: 3.856421244563535e-05, Final Batch Loss: 2.2192052711034194e-05
Epoch 4639, Loss: 1.

Epoch 4728, Loss: 0.00018760310194920748, Final Batch Loss: 8.873151818988845e-05
Epoch 4729, Loss: 0.00015429521590704098, Final Batch Loss: 0.00013481445785146207
Epoch 4730, Loss: 6.0857613334519556e-05, Final Batch Loss: 5.802227678941563e-05
Epoch 4731, Loss: 0.0001703473535599187, Final Batch Loss: 4.391609400045127e-05
Epoch 4732, Loss: 4.414806335262256e-05, Final Batch Loss: 2.0939269234077074e-06
Epoch 4733, Loss: 0.0002410912129562348, Final Batch Loss: 0.00018342252587899566
Epoch 4734, Loss: 0.00039652878604101716, Final Batch Loss: 7.161764187912922e-06
Epoch 4735, Loss: 0.0007836577497073449, Final Batch Loss: 0.0007400743197649717
Epoch 4736, Loss: 0.0002580004402261693, Final Batch Loss: 0.00020366182434372604
Epoch 4737, Loss: 2.7377353490010137e-05, Final Batch Loss: 2.2428152078646235e-05
Epoch 4738, Loss: 7.220726547529921e-05, Final Batch Loss: 6.0734397266060114e-05
Epoch 4739, Loss: 4.563540414892486e-06, Final Batch Loss: 2.99003227155481e-06
Epoch 4740, Loss: 

Epoch 4844, Loss: 9.415656222699909e-06, Final Batch Loss: 2.535747626097873e-06
Epoch 4845, Loss: 0.0005742817011196166, Final Batch Loss: 0.00027703167870640755
Epoch 4846, Loss: 1.1558293067537306e-05, Final Batch Loss: 1.3894401718062e-06
Epoch 4847, Loss: 2.6923916266241577e-05, Final Batch Loss: 7.387500772892963e-06
Epoch 4848, Loss: 1.1016149528586539e-05, Final Batch Loss: 8.741216333874036e-06
Epoch 4849, Loss: 0.0038904047978576273, Final Batch Loss: 0.0038589532487094402
Epoch 4850, Loss: 1.1488857353469939e-05, Final Batch Loss: 8.135365533235017e-06
Epoch 4851, Loss: 3.978940276283538e-05, Final Batch Loss: 6.163815669424366e-06
Epoch 4852, Loss: 3.272541317755895e-05, Final Batch Loss: 6.092816988712002e-07
Epoch 4853, Loss: 0.000299573965548916, Final Batch Loss: 2.2705376068188343e-06
Epoch 4854, Loss: 0.0003402732995709812, Final Batch Loss: 3.9521414691989776e-06
Epoch 4855, Loss: 2.1721593839174602e-05, Final Batch Loss: 5.3873200158705e-06
Epoch 4856, Loss: 2.66798

Epoch 4948, Loss: 1.959516112037818e-05, Final Batch Loss: 1.4883934454701375e-05
Epoch 4949, Loss: 6.9973755216778954e-06, Final Batch Loss: 4.769332008436322e-06
Epoch 4950, Loss: 0.0006368725428274047, Final Batch Loss: 3.356426759637543e-06
Epoch 4951, Loss: 0.00010268626920151291, Final Batch Loss: 1.2652878467633855e-05
Epoch 4952, Loss: 0.0001991934077523183, Final Batch Loss: 3.324319186503999e-05
Epoch 4953, Loss: 5.492302625498269e-05, Final Batch Loss: 1.281084587390069e-05
Epoch 4954, Loss: 0.00037339064527941446, Final Batch Loss: 0.0003724808630067855
Epoch 4955, Loss: 8.303749382321257e-05, Final Batch Loss: 7.236948295030743e-05
Epoch 4956, Loss: 0.00027133744356433454, Final Batch Loss: 0.0002683489874470979
Epoch 4957, Loss: 2.03126586484359e-06, Final Batch Loss: 3.7023909271738376e-07
Epoch 4958, Loss: 0.0003716269562907826, Final Batch Loss: 3.847493346142983e-08
Epoch 4959, Loss: 2.31641474783828e-05, Final Batch Loss: 1.742218591971323e-05
Epoch 4960, Loss: 0.000

In [10]:
softmax = nn.Softmax(dim = 1)
model.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[32  0  0]
 [ 1 39  0]
 [ 0  0 38]]
              precision    recall  f1-score   support

           0    0.96970   1.00000   0.98462        32
           1    1.00000   0.97500   0.98734        40
           2    1.00000   1.00000   1.00000        38

    accuracy                        0.99091       110
   macro avg    0.98990   0.99167   0.99065       110
weighted avg    0.99118   0.99091   0.99092       110



In [11]:
gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_Group_5_gen.param")
gen.eval()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): Linear(in_features=106, out_features=80, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Linear(in_features=80, out_features=60, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Linear(in_features=60, out_features=50, bias=True)
      (1): Dropout(p=0.1, inplace=False)
      (2): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU(inplace=True)
    )
    (3): Linear(in_features=50, out_features=46, bias=True)
    (4): Tanh()
  )
)

In [26]:
size = len(X_test)
latent_vectors = get_noise(size, 100)
act_vectors = get_act_matrix(size, 3)
usr_vectors = get_usr_matrix(size, 3)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach()

In [13]:
_, preds = torch.max(softmax(model(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(act_vectors[0], preds.cpu()))
print(metrics.classification_report(act_vectors[0], preds.cpu(), digits = 5))

[[35  0  0]
 [ 0 33  0]
 [ 0  0 42]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        35
           1    1.00000   1.00000   1.00000        33
           2    1.00000   1.00000   1.00000        42

    accuracy                        1.00000       110
   macro avg    1.00000   1.00000   1.00000       110
weighted avg    1.00000   1.00000   1.00000       110



# User Classifier

In [14]:
activities = [1, 3, 4]
users = [23, 25, 27]

X, y = start_data(activities, users, "Subject", sub_features, act_features)

In [15]:
for k in range(len(y)):
    if y[k] == 23:
        y[k] = 0
    elif y[k] == 25:
        y[k] = 1
    else:
        y[k] = 2

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)

model_subject = Classifier()
lr = 0.001
n_epochs = 5000
batch_size = 250

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_subject.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

In [17]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        
        optimizer.zero_grad()
        preds = model_subject(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')

Epoch 1, Loss: 2.3086363077163696, Final Batch Loss: 1.1559561491012573
Epoch 2, Loss: 2.3013386726379395, Final Batch Loss: 1.1542434692382812
Epoch 3, Loss: 2.2967309951782227, Final Batch Loss: 1.149670124053955
Epoch 4, Loss: 2.292755603790283, Final Batch Loss: 1.1495531797409058
Epoch 5, Loss: 2.2890398502349854, Final Batch Loss: 1.1501538753509521
Epoch 6, Loss: 2.2806206941604614, Final Batch Loss: 1.1259349584579468
Epoch 7, Loss: 2.2806702852249146, Final Batch Loss: 1.140572428703308
Epoch 8, Loss: 2.272704243659973, Final Batch Loss: 1.1240222454071045
Epoch 9, Loss: 2.269160509109497, Final Batch Loss: 1.1231893301010132
Epoch 10, Loss: 2.2675135135650635, Final Batch Loss: 1.1380356550216675
Epoch 11, Loss: 2.2658989429473877, Final Batch Loss: 1.1533607244491577
Epoch 12, Loss: 2.255954384803772, Final Batch Loss: 1.1186026334762573
Epoch 13, Loss: 2.255547285079956, Final Batch Loss: 1.1265558004379272
Epoch 14, Loss: 2.249314785003662, Final Batch Loss: 1.123370885848

Epoch 131, Loss: 0.883942723274231, Final Batch Loss: 0.4319545328617096
Epoch 132, Loss: 0.910086989402771, Final Batch Loss: 0.45880788564682007
Epoch 133, Loss: 0.9022880494594574, Final Batch Loss: 0.4420330822467804
Epoch 134, Loss: 0.8596119284629822, Final Batch Loss: 0.4077427387237549
Epoch 135, Loss: 0.8388789594173431, Final Batch Loss: 0.3867569863796234
Epoch 136, Loss: 0.8405241072177887, Final Batch Loss: 0.4078630208969116
Epoch 137, Loss: 0.8370073735713959, Final Batch Loss: 0.3953127861022949
Epoch 138, Loss: 0.8176299929618835, Final Batch Loss: 0.41398385167121887
Epoch 139, Loss: 0.8498306274414062, Final Batch Loss: 0.38221725821495056
Epoch 140, Loss: 0.815652459859848, Final Batch Loss: 0.41660013794898987
Epoch 141, Loss: 0.7639546394348145, Final Batch Loss: 0.3342606723308563
Epoch 142, Loss: 0.7943935096263885, Final Batch Loss: 0.38227570056915283
Epoch 143, Loss: 0.7749638259410858, Final Batch Loss: 0.37422987818717957
Epoch 144, Loss: 0.8343122005462646

Epoch 247, Loss: 0.34961308538913727, Final Batch Loss: 0.16480015218257904
Epoch 248, Loss: 0.3218728005886078, Final Batch Loss: 0.1408226490020752
Epoch 249, Loss: 0.36421744525432587, Final Batch Loss: 0.16180890798568726
Epoch 250, Loss: 0.3544015735387802, Final Batch Loss: 0.18981175124645233
Epoch 251, Loss: 0.33909010887145996, Final Batch Loss: 0.12542863190174103
Epoch 252, Loss: 0.4142443388700485, Final Batch Loss: 0.23956891894340515
Epoch 253, Loss: 0.3465155363082886, Final Batch Loss: 0.1538282036781311
Epoch 254, Loss: 0.330171599984169, Final Batch Loss: 0.189218670129776
Epoch 255, Loss: 0.3814755231142044, Final Batch Loss: 0.17784714698791504
Epoch 256, Loss: 0.33937792479991913, Final Batch Loss: 0.1832485944032669
Epoch 257, Loss: 0.2855118662118912, Final Batch Loss: 0.1592671424150467
Epoch 258, Loss: 0.38706086575984955, Final Batch Loss: 0.21445602178573608
Epoch 259, Loss: 0.32787613570690155, Final Batch Loss: 0.1635766476392746
Epoch 260, Loss: 0.33205658

Epoch 364, Loss: 0.2477797567844391, Final Batch Loss: 0.16083915531635284
Epoch 365, Loss: 0.26073676347732544, Final Batch Loss: 0.12189500033855438
Epoch 366, Loss: 0.3366616815328598, Final Batch Loss: 0.13154369592666626
Epoch 367, Loss: 0.23138292133808136, Final Batch Loss: 0.12063883244991302
Epoch 368, Loss: 0.24936895072460175, Final Batch Loss: 0.15228338539600372
Epoch 369, Loss: 0.23603935539722443, Final Batch Loss: 0.11188054084777832
Epoch 370, Loss: 0.2693890854716301, Final Batch Loss: 0.12496484071016312
Epoch 371, Loss: 0.19917602837085724, Final Batch Loss: 0.10790852457284927
Epoch 372, Loss: 0.2761834114789963, Final Batch Loss: 0.1529269516468048
Epoch 373, Loss: 0.20394328981637955, Final Batch Loss: 0.11632151156663895
Epoch 374, Loss: 0.21282906085252762, Final Batch Loss: 0.09961779415607452
Epoch 375, Loss: 0.23510070145130157, Final Batch Loss: 0.10419423878192902
Epoch 376, Loss: 0.2470843344926834, Final Batch Loss: 0.12165090441703796
Epoch 377, Loss: 0

Epoch 488, Loss: 0.19346727430820465, Final Batch Loss: 0.08086694031953812
Epoch 489, Loss: 0.18550102412700653, Final Batch Loss: 0.07841479033231735
Epoch 490, Loss: 0.2242213785648346, Final Batch Loss: 0.1361234337091446
Epoch 491, Loss: 0.19309549033641815, Final Batch Loss: 0.06857980787754059
Epoch 492, Loss: 0.18466821312904358, Final Batch Loss: 0.07735401391983032
Epoch 493, Loss: 0.21860643476247787, Final Batch Loss: 0.12872900068759918
Epoch 494, Loss: 0.19851134717464447, Final Batch Loss: 0.07456520944833755
Epoch 495, Loss: 0.1934514418244362, Final Batch Loss: 0.11348912864923477
Epoch 496, Loss: 0.2126493975520134, Final Batch Loss: 0.09168587625026703
Epoch 497, Loss: 0.15207058191299438, Final Batch Loss: 0.08889327198266983
Epoch 498, Loss: 0.14644037187099457, Final Batch Loss: 0.06304334849119186
Epoch 499, Loss: 0.20811446756124496, Final Batch Loss: 0.08408410847187042
Epoch 500, Loss: 0.2512208893895149, Final Batch Loss: 0.13020217418670654
Epoch 501, Loss: 

...

Epoch 4430, Loss: 0.09810440614819527, Final Batch Loss: 0.04515647888183594
Epoch 4431, Loss: 0.12482190504670143, Final Batch Loss: 0.08408983051776886
Epoch 4432, Loss: 0.08158440515398979, Final Batch Loss: 0.04420182481408119
Epoch 4433, Loss: 0.07511194795370102, Final Batch Loss: 0.034141905605793
Epoch 4434, Loss: 0.08444449678063393, Final Batch Loss: 0.038043245673179626
Epoch 4435, Loss: 0.10995877161622047, Final Batch Loss: 0.0856214240193367
Epoch 4436, Loss: 0.08077893033623695, Final Batch Loss: 0.03571850061416626
Epoch 4437, Loss: 0.0752466693520546, Final Batch Loss: 0.017343640327453613
Epoch 4438, Loss: 0.061830466613173485, Final Batch Loss: 0.03391016647219658
Epoch 4439, Loss: 0.07500125467777252, Final Batch Loss: 0.03705384582281113
Epoch 4440, Loss: 0.0644195806235075, Final Batch Loss: 0.034766875207424164
Epoch 4441, Loss: 0.06513172015547752, Final Batch Loss: 0.034754037857055664
Epoch 4442, Loss: 0.08190420642495155, Final Batch Loss: 0.04441384226083755

Epoch 4542, Loss: 0.0902150496840477, Final Batch Loss: 0.03705102205276489
Epoch 4543, Loss: 0.08936554193496704, Final Batch Loss: 0.055725663900375366
Epoch 4544, Loss: 0.09568949416279793, Final Batch Loss: 0.04728781431913376
Epoch 4545, Loss: 0.09759333916008472, Final Batch Loss: 0.03046363778412342
Epoch 4546, Loss: 0.11945271119475365, Final Batch Loss: 0.03770764544606209
Epoch 4547, Loss: 0.09659566357731819, Final Batch Loss: 0.04280709847807884
Epoch 4548, Loss: 0.060398077592253685, Final Batch Loss: 0.02466401644051075
Epoch 4549, Loss: 0.07322771847248077, Final Batch Loss: 0.03185039013624191
Epoch 4550, Loss: 0.07403062656521797, Final Batch Loss: 0.032986901700496674
Epoch 4551, Loss: 0.09677747264504433, Final Batch Loss: 0.05195952579379082
Epoch 4552, Loss: 0.09083596244454384, Final Batch Loss: 0.050977617502212524
Epoch 4553, Loss: 0.08433158695697784, Final Batch Loss: 0.05309443548321724
Epoch 4554, Loss: 0.0872626043856144, Final Batch Loss: 0.048230059444904

Epoch 4656, Loss: 0.13560800813138485, Final Batch Loss: 0.021599529311060905
Epoch 4657, Loss: 0.06403588503599167, Final Batch Loss: 0.032964836806058884
Epoch 4658, Loss: 0.07729456201195717, Final Batch Loss: 0.04352713003754616
Epoch 4659, Loss: 0.08032851479947567, Final Batch Loss: 0.05406048521399498
Epoch 4660, Loss: 0.143990408629179, Final Batch Loss: 0.09523195028305054
Epoch 4661, Loss: 0.0809735357761383, Final Batch Loss: 0.026216834783554077
Epoch 4662, Loss: 0.08444879949092865, Final Batch Loss: 0.023748990148305893
Epoch 4663, Loss: 0.11618665605783463, Final Batch Loss: 0.07108257710933685
Epoch 4664, Loss: 0.1242901012301445, Final Batch Loss: 0.06766227632761002
Epoch 4665, Loss: 0.06420821137726307, Final Batch Loss: 0.025782691314816475
Epoch 4666, Loss: 0.13653556630015373, Final Batch Loss: 0.09447871893644333
Epoch 4667, Loss: 0.0693109855055809, Final Batch Loss: 0.04221910983324051
Epoch 4668, Loss: 0.09239742532372475, Final Batch Loss: 0.03666151314973831

Epoch 4774, Loss: 0.06263037025928497, Final Batch Loss: 0.021789751946926117
Epoch 4775, Loss: 0.06619792059063911, Final Batch Loss: 0.031162269413471222
Epoch 4776, Loss: 0.07320603169500828, Final Batch Loss: 0.02126876823604107
Epoch 4777, Loss: 0.07692956738173962, Final Batch Loss: 0.030475763604044914
Epoch 4778, Loss: 0.09063076972961426, Final Batch Loss: 0.03612188622355461
Epoch 4779, Loss: 0.08791397139430046, Final Batch Loss: 0.02851647138595581
Epoch 4780, Loss: 0.07168367877602577, Final Batch Loss: 0.038825623691082
Epoch 4781, Loss: 0.07146849296987057, Final Batch Loss: 0.04274468868970871
Epoch 4782, Loss: 0.08572696894407272, Final Batch Loss: 0.043314095586538315
Epoch 4783, Loss: 0.06379630044102669, Final Batch Loss: 0.040975283831357956
Epoch 4784, Loss: 0.11943469196557999, Final Batch Loss: 0.03196951001882553
Epoch 4785, Loss: 0.07722742110490799, Final Batch Loss: 0.051417794078588486
Epoch 4786, Loss: 0.08120905980467796, Final Batch Loss: 0.0286005213856

Epoch 4903, Loss: 0.07800963893532753, Final Batch Loss: 0.03488348051905632
Epoch 4904, Loss: 0.11720189452171326, Final Batch Loss: 0.060910455882549286
Epoch 4905, Loss: 0.10188327729701996, Final Batch Loss: 0.04927203804254532
Epoch 4906, Loss: 0.11300739273428917, Final Batch Loss: 0.041119735687971115
Epoch 4907, Loss: 0.09167656116187572, Final Batch Loss: 0.029775256291031837
Epoch 4908, Loss: 0.12045059353113174, Final Batch Loss: 0.07559816539287567
Epoch 4909, Loss: 0.12708278745412827, Final Batch Loss: 0.05700545758008957
Epoch 4910, Loss: 0.10459793731570244, Final Batch Loss: 0.03836965933442116
Epoch 4911, Loss: 0.10952742397785187, Final Batch Loss: 0.07665973901748657
Epoch 4912, Loss: 0.08046027086675167, Final Batch Loss: 0.02786836586892605
Epoch 4913, Loss: 0.11261813715100288, Final Batch Loss: 0.05207470431923866
Epoch 4914, Loss: 0.09670998901128769, Final Batch Loss: 0.049401577562093735
Epoch 4915, Loss: 0.09036281332373619, Final Batch Loss: 0.0550844110548

In [18]:
softmax = nn.Softmax(dim = 1)
model_subject.eval()
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model_subject(features.float())), dim = 1)
    print(metrics.confusion_matrix((labels).cpu(), preds.cpu()))
    print(metrics.classification_report((labels).cpu(), preds.cpu(), digits = 5))

[[24  2  0]
 [ 1 42  0]
 [ 0  0 41]]
              precision    recall  f1-score   support

           0    0.96000   0.92308   0.94118        26
           1    0.95455   0.97674   0.96552        43
           2    1.00000   1.00000   1.00000        41

    accuracy                        0.97273       110
   macro avg    0.97152   0.96661   0.96890       110
weighted avg    0.97278   0.97273   0.97262       110



In [27]:
_, preds = torch.max(softmax(model_subject(fake_features.float())), dim = 1)
print(metrics.confusion_matrix(usr_vectors[0], preds.cpu()))
print(metrics.classification_report(usr_vectors[0], preds.cpu(), digits = 5))

[[38  0  0]
 [ 0 38  0]
 [ 0  0 34]]
              precision    recall  f1-score   support

           0    1.00000   1.00000   1.00000        38
           1    1.00000   1.00000   1.00000        38
           2    1.00000   1.00000   1.00000        34

    accuracy                        1.00000       110
   macro avg    1.00000   1.00000   1.00000       110
weighted avg    1.00000   1.00000   1.00000       110

